In [1]:
from collections import Counter
import pandas as pd
import zipfile
import json
import os

In [2]:
root = "."
pathogen_code = 'mtuberculosis'

In [3]:
# Load cleaned assays
ASSAYS_CLEANED = pd.read_csv(os.path.join(root, "..", "output", pathogen_code, "assays_cleaned.csv"))

# Define PATH to parameters
PATH_TO_PARAMETERS = pd.read_csv(os.path.join(root, "..", "output", pathogen_code, 'assays_parameters.csv'))

# Merge tables
OUT = ASSAYS_CLEANED.merge(PATH_TO_PARAMETERS, on=['assay_id', 'activity_type', 'unit'], how='inner', validate='1:1')

In [4]:
Counter(ASSAYS_CLEANED['target_type'])

Counter({'ORGANISM': 8663,
         'SINGLE PROTEIN': 954,
         'UNCHECKED': 797,
         'PROTEIN COMPLEX': 84,
         'SUBCELLULAR': 17,
         'NON-MOLECULAR': 13,
         'ADMET': 2,
         'NO TARGET': 1,
         'PROTEIN FAMILY': 1})

In [5]:
len(OUT[OUT['target_type'] == 'UNCHECKED'])

40

In [6]:
len(OUT[(OUT['target_type'] == 'UNCHECKED') & (OUT['target_type_curated'] == 'DISCARDED')])

1

In [7]:
len(OUT[(OUT['target_type'] == 'UNCHECKED') & (OUT['target_type_curated'] == 'ORGANISM')])

14

In [8]:
len(OUT[(OUT['target_type'] == 'UNCHECKED') & (OUT['target_type_curated'] == 'SINGLE PROTEIN')])

25

In [9]:
OUT[(OUT['target_type'] == 'UNCHECKED') & (OUT['target_type_curated'].isin(['ORGANISM', 'SINGLE PROTEIN']) == False)]

,assay_id,assay_type,assay_organism,doc_chembl_id,target_type,target_chembl_id,target_organism,bao_label,source_label,activity_type,...,direction,organism_curated,target_type_curated,target_name_curated,target_chembl_id_curated,strain,atcc_id,mutations,known_drug_resistances,media
25,CHEMBL1738421,F,NaN,CHEMBL1201862,UNCHECKED,CHEMBL612545,NaN,cell-based format,PUBCHEM_BIOASSAY,CC50,...,-1.0,Mycobacterium tuberculosis,DISCARDED,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
OUT

,assay_id,assay_type,assay_organism,doc_chembl_id,target_type,target_chembl_id,target_organism,bao_label,source_label,activity_type,...,direction,organism_curated,target_type_curated,target_name_curated,target_chembl_id_curated,strain,atcc_id,mutations,known_drug_resistances,media
0,CHEMBL4649948,F,Mycobacterium tuberculosis,CHEMBL3988442,UNCHECKED,CHEMBL612545,NaN,organism-based format,GATES_LIBRARY,PERCENTEFFECT,...,1.0,Mycobacterium tuberculosis,ORGANISM,Mycobacterium tuberculosis,NaN,NaN,NaN,NaN,NaN,DPPC; cholesterol; tyloxapol based media
1,CHEMBL4649949,F,Mycobacterium tuberculosis,CHEMBL3988442,UNCHECKED,CHEMBL612545,NaN,organism-based format,GATES_LIBRARY,PERCENTEFFECT,...,1.0,Mycobacterium tuberculosis,ORGANISM,NaN,NaN,NaN,NaN,NaN,NaN,7H9; glucose tyloxapol based media
2,CHEMBL4649971,F,Mycobacterium tuberculosis,CHEMBL3988442,ORGANISM,CHEMBL360,Mycobacterium tuberculosis,organism-based format,GATES_LIBRARY,PERCENTEFFECT,...,1.0,Mycobacterium tuberculosis,ORGANISM,Mycobacterium tuberculosis,NaN,NaN,NaN,NaN,NaN,NaN
3,CHEMBL4649972,F,Mycobacterium tuberculosis,CHEMBL3988442,PROTEIN COMPLEX,CHEMBL4662931,Mycobacterium tuberculosis (strain ATCC 25618 ...,assay format,GATES_LIBRARY,PERCENTEFFECT,...,1.0,Mycobacterium tuberculosis,PROTEIN COMPLEX,ClpP1P2,NaN,H37Rv,ATCC 25618,NaN,NaN,NaN
4,CHEMBL4649941,F,Mycobacterium tuberculosis,CHEMBL3988442,SINGLE PROTEIN,CHEMBL4662928,Mycobacterium tuberculosis (strain ATCC 25618 ...,assay format,GATES_LIBRARY,PERCENTEFFECT,...,1.0,Mycobacterium tuberculosis,SINGLE PROTEIN,MtCoaBC,NaN,H37Rv,ATCC 25618,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,CHEMBL3758008,B,Mycobacterium tuberculosis,CHEMBL3751721,UNCHECKED,CHEMBL612545,NaN,assay format,LITERATURE,INHIBITION,...,1.0,Mycobacterium tuberculosis,SINGLE PROTEIN,DNA topoisomerase 1,NaN,NaN,NaN,NaN,NaN,NaN
494,CHEMBL837765,B,Mycobacterium tuberculosis,CHEMBL1138787,SINGLE PROTEIN,CHEMBL4450,Mycobacterium tuberculosis,assay format,LITERATURE,IC50,...,-1.0,Mycobacterium tuberculosis,SINGLE PROTEIN,UDP-galactose mutase,NaN,H37Rv,NaN,NaN,NaN,NaN
495,CHEMBL5108933,F,Mycobacterium tuberculosis,CHEMBL5107963,ORGANISM,CHEMBL360,Mycobacterium tuberculosis,organism-based format,LITERATURE,MIC,...,-1.0,Mycobacterium tuberculosis,ORGANISM,NaN,NaN,H37Rv,NaN,NaN,NaN,NaN
496,CHEMBL4003019,F,Mycobacterium tuberculosis,CHEMBL4002566,ORGANISM,CHEMBL360,Mycobacterium tuberculosis,assay format,LITERATURE,MIC,...,-1.0,Mycobacterium tuberculosis,ORGANISM,NaN,NaN,CNCTC My 331/88,NaN,NaN,NaN,NaN
